In [2]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

def generate_sample_news(n = 10,start_date = None):
    if start_date is None:
        start_date = datetime.now()
        
    companies = ['Apple','Microsoft','Tesla','Google','Amazon']
    sentiments = ['Positive','Neutral','Negative']
    data = {
        'timestamp': [start_date + timedelta(hours=i) for i in range(n)],
        'company': np.random.choice(companies, n),
        'headline': [f'News headline {i}' for i in range(n)],
        'sentiment': np.random.choice(sentiments, n),
        'sentiment_score': np.random.uniform(-1, 1, n)
    }

    return pd.DataFrame(data)

# Generate first batch of news
df_batch1 = generate_sample_news(10)
print("First batch of news:")
print(df_batch1.head())
print(f"\nData types:\n{df_batch1.dtypes}")

First batch of news:
                   timestamp company         headline sentiment  \
0 2026-01-07 18:53:44.717707   Apple  News headline 0  Positive   
1 2026-01-07 19:53:44.717707   Tesla  News headline 1  Positive   
2 2026-01-07 20:53:44.717707  Amazon  News headline 2  Positive   
3 2026-01-07 21:53:44.717707  Google  News headline 3  Negative   
4 2026-01-07 22:53:44.717707   Tesla  News headline 4  Positive   

   sentiment_score  
0         0.847145  
1         0.009470  
2         0.332856  
3        -0.181164  
4        -0.080321  

Data types:
timestamp          datetime64[ns]
company                    object
headline                   object
sentiment                  object
sentiment_score           float64
dtype: object


#### Creating SQLite Database

In [3]:
db_path = 'news.db'
conn = sqlite3.connect(db_path)

df_batch1.to_sql('news_data', conn, if_exists='replace', index=False)

conn.close()
print(f"Saved {len(df_batch1)} records to {db_path}")

Saved 10 records to news.db


In [4]:
df_batch2 = generate_sample_news(n = 20,start_date = datetime.now() + timedelta(days=2))

conn = sqlite3.connect(db_path)
df_batch2.to_sql('news_data',conn,if_exists='append',index=False)
conn.close()

print(f"Appended {len(df_batch2)} more records")

Appended 20 more records


#### Reading the data from SQL DB

In [5]:
conn = sqlite3.connect(db_path)
df_from_db = pd.read_sql_query('Select * from news_data',conn)
conn.close()

print(f"Total records in database: {len(df_from_db)}")
print(f"\nData types preserved:\n{df_from_db.dtypes}")
print(f"\nFirst few rows:\n{df_from_db.head()}")
print(f"\nLast few rows:\n{df_from_db.tail()}")

Total records in database: 30

Data types preserved:
timestamp           object
company             object
headline            object
sentiment           object
sentiment_score    float64
dtype: object

First few rows:
                    timestamp company         headline sentiment  \
0  2026-01-07 18:53:44.717707   Apple  News headline 0  Positive   
1  2026-01-07 19:53:44.717707   Tesla  News headline 1  Positive   
2  2026-01-07 20:53:44.717707  Amazon  News headline 2  Positive   
3  2026-01-07 21:53:44.717707  Google  News headline 3  Negative   
4  2026-01-07 22:53:44.717707   Tesla  News headline 4  Positive   

   sentiment_score  
0         0.847145  
1         0.009470  
2         0.332856  
3        -0.181164  
4        -0.080321  

Last few rows:
                     timestamp    company          headline sentiment  \
25  2026-01-10 10:02:15.848311  Microsoft  News headline 15   Neutral   
26  2026-01-10 11:02:15.848311  Microsoft  News headline 16   Neutral   
27  2026-01

In [7]:
conn = sqlite3.connect(db_path)

apple_news = pd.read_sql_query("""Select * from news_data where company = 'Apple' order by timestamp DESC""",conn)
print(f"Apple news count: {len(apple_news)}")
print(apple_news)

avg_sentiment = pd.read_sql_query("""select company, avg(sentiment_score) as avg_score from news_data group by company order by avg_score desc""",conn)
print(f"\nAverage sentiment by company:\n{avg_sentiment}")

conn.close()

Apple news count: 2
                    timestamp company          headline sentiment  \
0  2026-01-10 05:02:15.848311   Apple  News headline 10   Neutral   
1  2026-01-07 18:53:44.717707   Apple   News headline 0  Positive   

   sentiment_score  
0         0.761446  
1         0.847145  

Average sentiment by company:
     company  avg_score
0      Apple   0.804295
1  Microsoft   0.303287
2      Tesla  -0.012246
3     Amazon  -0.085577
4     Google  -0.179158


#### Implementation of Parquet

In [9]:
import pyarrow as pa
import pyarrow.parquet as pq
import os

parquet_file = 'news.parquet'
df_batch1.to_parquet(parquet_file, engine='pyarrow', index=False)

print(f"Saved {len(df_batch1)} records to {parquet_file}")

Saved 10 records to news.parquet


In [10]:
if os.path.exists(parquet_file):
    existing_df = pd.read_parquet(parquet_file)
    combined_df = pd.concat([existing_df, df_batch2], ignore_index=True)
else:
    combined_df = df_batch2

combined_df.to_parquet(parquet_file, engine='pyarrow', index=False)
print(f"Appended data. Total records: {len(combined_df)}")

Appended data. Total records: 30


#### Reading from a parquet file

In [11]:
df_from_parquet = pd.read_parquet(parquet_file)

print(f"Total records: {len(df_from_parquet)}")
print(f"\nData types preserved:\n{df_from_parquet.dtypes}")
print(f"\nFirst few rows:\n{df_from_parquet.head()}")

Total records: 30

Data types preserved:
timestamp          datetime64[ns]
company                    object
headline                   object
sentiment                  object
sentiment_score           float64
dtype: object

First few rows:
                   timestamp company         headline sentiment  \
0 2026-01-07 18:53:44.717707   Apple  News headline 0  Positive   
1 2026-01-07 19:53:44.717707   Tesla  News headline 1  Positive   
2 2026-01-07 20:53:44.717707  Amazon  News headline 2  Positive   
3 2026-01-07 21:53:44.717707  Google  News headline 3  Negative   
4 2026-01-07 22:53:44.717707   Tesla  News headline 4  Positive   

   sentiment_score  
0         0.847145  
1         0.009470  
2         0.332856  
3        -0.181164  
4        -0.080321  


In [12]:
csv_file = 'news.csv'
combined_df.to_csv(csv_file, index=False)

csv_size = os.path.getsize(csv_file) / 1024  # KB
parquet_size = os.path.getsize(parquet_file) / 1024  # KB
db_size = os.path.getsize(db_path) / 1024  # KB

print(f"File size comparison:")
print(f"CSV: {csv_size:.2f} KB")
print(f"Parquet: {parquet_size:.2f} KB")
print(f"SQLite: {db_size:.2f} KB")

File size comparison:
CSV: 2.35 KB
Parquet: 4.04 KB
SQLite: 8.00 KB
